In [1]:
!pip install graphrole

In [18]:
import pandas as pd
import snap
from src.load_and_save import load_networkx_directed_graph, save_list, load_list
from graphrole import RecursiveFeatureExtractor, RoleExtractor
import numpy as np
import os.path
import networkx as nx

In [6]:
followers_reduced_path = "data/followers_reduced.edgelist"
feature_extraction_path = "data/feature_extractions.csv"

In [29]:
followers_reduced_graph = load_networkx_directed_graph(followers_reduced_path, weighted=True)

# Extracting features

In [30]:
followers_feature_extractor = RecursiveFeatureExtractor(followers_reduced_graph)
followers_features = followers_feature_extractor.extract_features()
followers_features

,in_degree(mean)(mean)(mean)(mean)(mean),external_edges(mean)(mean)(mean)(mean),in_degree(mean)(mean)(mean)(mean),total_degree(mean)(mean)(mean)(mean),external_edges(mean)(mean)(mean),in_degree(mean)(mean)(mean),total_degree(mean)(mean)(mean),external_edges(mean)(mean),in_degree(mean)(mean),internal_edges(mean)(mean),...,internal_edges(mean),internal_edges(sum),out_degree(mean),total_degree(mean),total_degree(sum),external_edges,in_degree,internal_edges,out_degree,total_degree
2,67.311569,696.271774,69.302801,97.977370,711.971016,71.279955,100.573404,738.502428,77.790099,345.177058,...,349.650000,13986.0,30.950000,110.000000,4400.0,960,134,318,40,174
3,69.174312,751.957848,72.368114,103.087618,753.662990,68.568590,99.466422,741.620834,68.788832,319.868477,...,236.846154,3079.0,24.846154,93.846154,1220.0,272,17,64,13,30
4,68.749972,733.515900,70.025479,100.047430,764.825527,71.949679,103.162715,816.826325,74.107427,380.311763,...,327.074766,34997.0,31.411215,91.813084,9824.0,1845,184,1623,107,291
8,71.954880,818.238002,73.871078,106.951570,889.443458,74.120415,109.678580,1017.350219,79.006985,537.200243,...,607.153846,15786.0,42.884615,126.615385,3292.0,898,230,243,26,256
9,68.568127,678.910798,69.085264,97.159414,667.227771,68.720924,95.761450,683.976199,64.026301,327.760074,...,231.615385,3011.0,24.384615,99.692308,1296.0,290,21,40,13,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264163,73.682534,709.663196,76.205898,105.947203,720.637546,76.751150,107.140605,677.749947,84.499480,287.946949,...,320.750000,2566.0,30.375000,111.375000,891.0,234,0,17,8,8
288958,69.315284,617.816862,71.350734,97.500146,595.620523,75.347868,100.872546,579.307674,74.636241,285.508628,...,220.272727,4846.0,24.636364,102.545455,2256.0,406,0,158,22,22
312038,70.345717,520.599441,68.652426,91.966122,394.984098,62.948538,82.223951,220.882381,45.368571,67.102381,...,52.000000,260.0,13.800000,26.800000,134.0,54,0,20,5,5
385575,75.822063,664.310685,79.778940,107.996515,653.590996,81.382773,109.907157,624.543310,93.884281,311.116866,...,204.666667,1228.0,25.333333,96.833333,581.0,134,0,24,6,6


In [44]:
# DataFrame to manipulate
followers_features_clustering = followers_features[["external_edges", "in_degree", "internal_edges", "out_degree", "total_degree"]].copy()

In [45]:
followers_features_clustering

,external_edges,in_degree,internal_edges,out_degree,total_degree
2,960,134,318,40,174
3,272,17,64,13,30
4,1845,184,1623,107,291
8,898,230,243,26,256
9,290,21,40,13,34
...,...,...,...,...,...
264163,234,0,17,8,8
288958,406,0,158,22,22
312038,54,0,20,5,5
385575,134,0,24,6,6


# Adding extra features

## Betweeness centrality

In [41]:
# Loading graph with snap
snap_followers_graph = snap.LoadEdgeList(snap.TNGraph, followers_reduced_path, 0, 1)

In [34]:
# Extrancting betweeness centrality
Nodes, _ = snap_followers_graph.GetBetweennessCentr(1.0, False)

In [46]:
# Collecting node number and betweeness centrality
list_nodes = []
list_betwCentrality = []
for node in Nodes:
    list_nodes.append(node)
    list_betwCentrality.append(Nodes[node])
    
followers_features_clustering["Betweeness Centrality"] = list_betwCentrality
followers_features_clustering

,external_edges,in_degree,internal_edges,out_degree,total_degree,Betweeness Centrality
2,960,134,318,40,174,5308.949046
3,272,17,64,13,30,601.957930
4,1845,184,1623,107,291,556.217951
8,898,230,243,26,256,15106.927788
9,290,21,40,13,34,7452.897306
...,...,...,...,...,...,...
264163,234,0,17,8,8,0.000000
288958,406,0,158,22,22,1096.670337
312038,54,0,20,5,5,35.264584
385575,134,0,24,6,6,6.902604


## K-core

In [47]:
followers_reduced_noloops_graph = followers_reduced_graph.copy()
followers_reduced_graph.remove_edges_from(nx.selfloop_edges(followers_reduced_graph))
followers_features_clustering["k-core"] = nx.core_number(followers_reduced_graph)
followers_features_clustering

,external_edges,in_degree,internal_edges,out_degree,total_degree,Betweeness Centrality,k-core
2,960,134,318,40,174,5308.949046,2
3,272,17,64,13,30,601.957930,14
4,1845,184,1623,107,291,556.217951,25
8,898,230,243,26,256,15106.927788,26
9,290,21,40,13,34,7452.897306,27
...,...,...,...,...,...,...,...
264163,234,0,17,8,8,0.000000,1249
288958,406,0,158,22,22,1096.670337,1259
312038,54,0,20,5,5,35.264584,1255
385575,134,0,24,6,6,6.902604,19787


# Extracting roles

In [48]:
# Using RoleX to extract roles
role_extractor = RoleExtractor()
role_extractor.extract_role_factors(followers_features_clustering)
roles = role_extractor.roles

c:\users\98ric\appdata\local\programs\python\python38\lib\site-packages\graphrole\roles\description_length.py:56: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(np.where(vec1 != 0, vec1 * np.log(vec1 / vec2) - vec1 + vec2, 0))
c:\users\98ric\appdata\local\programs\python\python38\lib\site-packages\graphrole\roles\description_length.py:56: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(np.where(vec1 != 0, vec1 * np.log(vec1 / vec2) - vec1 + vec2, 0))
c:\users\98ric\appdata\local\programs\python\python38\lib\site-packages\graphrole\roles\description_length.py:56: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(np.where(vec1 != 0, vec1 * np.log(vec1 / vec2) - vec1 + vec2, 0))
c:\users\98ric\appdata\local\programs\python\python38\lib\site-packages\graphrole\roles\description_length.py:56: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(np.where(vec1 != 0, vec1 * np.log(vec1 / vec2) - vec1 + vec2, 0))


c:\users\98ric\appdata\local\programs\python\python38\lib\site-packages\graphrole\roles\description_length.py:56: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(np.where(vec1 != 0, vec1 * np.log(vec1 / vec2) - vec1 + vec2, 0))
c:\users\98ric\appdata\local\programs\python\python38\lib\site-packages\graphrole\roles\description_length.py:56: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(np.where(vec1 != 0, vec1 * np.log(vec1 / vec2) - vec1 + vec2, 0))
c:\users\98ric\appdata\local\programs\python\python38\lib\site-packages\graphrole\roles\description_length.py:56: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(np.where(vec1 != 0, vec1 * np.log(vec1 / vec2) - vec1 + vec2, 0))
c:\users\98ric\appdata\local\programs\python\python38\lib\site-packages\graphrole\roles\description_length.py:56: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(np.where(vec1 != 0, vec1 * np.log(vec1 / vec2) - vec1 + vec2, 0))


In [49]:
# List of clusters
list_roles = list(roles.values())

In [50]:
# Counting number of nodes in each cluster
clusters, counts = np.unique(list_roles, return_counts=True)
for cluster, count in zip(clusters, counts):
    print(cluster, count)

role_0 728
role_1 108
role_2 36
role_3 73
role_4 124


In [51]:
# Adding cluster column
cluster_df = followers_features_clustering.copy()
cluster_df['cluster'] = list_roles

In [52]:
followers_features_clustering = cluster_df.copy()

In [53]:
followers_features_clustering

,external_edges,in_degree,internal_edges,out_degree,total_degree,Betweeness Centrality,k-core,cluster
2,960,134,318,40,174,5308.949046,2,role_3
3,272,17,64,13,30,601.957930,14,role_0
4,1845,184,1623,107,291,556.217951,25,role_0
8,898,230,243,26,256,15106.927788,26,role_3
9,290,21,40,13,34,7452.897306,27,role_3
...,...,...,...,...,...,...,...,...
264163,234,0,17,8,8,0.000000,1249,role_0
288958,406,0,158,22,22,1096.670337,1259,role_0
312038,54,0,20,5,5,35.264584,1255,role_0
385575,134,0,24,6,6,6.902604,19787,role_1


# Save to file

In [19]:
# Saving data frame with roles/clusters included
followers_features_clustering.to_csv(feature_extraction_path)